In [ ]:
import pandas as pd
import akshare as ak
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
sns.set(font='SimHei')
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 
df = ak.stock_zh_a_spot_em()
df.to_excel('example.xlsx', sheet_name='Sheet1', index=False)

In [ ]:
import pandas as pd
import akshare as ak
df=ak.stock_zh_a_hist(symbol='688484', period='daily', start_date='20230601', end_date='20240601', adjust='qfq')
df.to_csv('688484cba.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
lit=['收盘']
data=df[lit]
result = adfuller(data)
print("Test Statistic: %f" % result[0])
print("p-value: %f" % result[1])
print("No. of lags used: %f" % result[2])
print("Number of observations used: %f" % result[3])
print("critical value 1%%: %f" % result[4]["1%"])
print("critical value 5%%: %f" % result[4]["5%"])
print("critical value 10%%: %f" % result[4]["10%"])
if result[1] > 0.05:
    diff_data = data.diff().dropna()
    plt.figure()
    plt.plot(data, label='Original')
    plt.plot(diff_data, label='Differenced')
    plt.legend()
    data = diff_data
else:
    plt.figure()
    plt.plot(data, label='Original')
    plt.legend()

In [ ]:
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
plt.figure()
plt.plot(scaled_data, label='Scaled')
plt.legend()

In [ ]:
aic_values = {}
for p in range(6):
    for q in range(6):
        try:
            model = ARIMA(scaled_data, order=(p, 1, q))
            result = model.fit()
            aic_values[(p, 1, q)] = result.aic
        except:
            continue
min_aic = min(aic_values, key=aic_values.get)
print("min aci:", min_aic)
model = ARIMA(scaled_data, order=min_aic)
result = model.fit()
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(scaled_data, ax=axes[0])
plot_pacf(scaled_data, ax=axes[1])
plt.show()

In [ ]:
stock_file = '688484cba.csv'
df = pd.read_csv(stock_file, index_col='日期', parse_dates=True)
df

In [ ]:
stock_week = df['收盘']
stock_train = stock_week['2023-06-01':'2024-05-01']
stock_actual=stock_week['2024-05-01':'2024-06-01']
stock_train.plot(figsize=(15,6))
plt.legend()
plt.title('Stock Close')
sns.despine()

In [ ]:
stock_actual=stock_week['2024-05-01':'2024-06-01']

In [ ]:
# 将时间序列进行差分并确定参数d
# 一阶差分
stock_diff_1 = stock_train.diff()
stock_diff_1.dropna(inplace=True)
# 二阶差分
stock_diff_2 = stock_diff_1.diff()
stock_diff_2.dropna(inplace=True)
plt.figure(figsize=(12,6))

plt.subplot(2,1,2)
plt.plot(stock_diff_2)
plt.title('Second Differential')
plt.show()

### 拟合ARIMA模型


In [ ]:
model = sm.tsa.ARIMA(stock_train, order=(2, 1, 4))
result = model.fit()

In [ ]:
stock_future = stock_week['2024-05-01':]

In [ ]:
# 使用该模型进行预测
forecast = result.forecast(len(stock_future))
forecast.index=stock_future.index

In [ ]:
forecast

In [ ]:
plt.xticks(rotation=45)
plt.plot(forecast,label='Prediction')
plt.plot(stock_actual,label='Actual Value')
plt.plot(stock_train,label='Train')
plt.legend()
plt.show()

In [ ]:
plt.xticks(rotation=45)
plt.plot(forecast,label='Prediction')
plt.plot(stock_actual,label='Actual Value')
plt.legend()
plt.show()